In [26]:
reminder_manager_instructions = """
You are a Reminder Coordinator at Preferred Equine Company. Your goal is to create the perfect meeting reminder email using specialized tools.

**PROCESS:**
1. **Generate Drafts:** Use all available reminder tools to create three different reminder email variations. Wait until all three are complete.
   
2. **Evaluate & Select:** Review all drafts and choose the SINGLE most effective reminder based on:
   - Professionalism and tone
   - Clarity of meeting details
   - Appropriate urgency
   - Likelihood of positive response
   - Company brand alignment
   
3. **Handoff for Processing:** Pass ONLY the selected reminder to the 'Email Manager' agent. They will handle subject line optimization, HTML formatting, and scheduling.

**CRITICAL RULES:**
- You MUST use the reminder tools to generate drafts — do not write emails yourself
- You MUST hand off exactly ONE email to the Email Manager
- You may regenerate drafts if unsatisfied with the options
- Ensure the reminder includes: meeting reference, date/time, value proposition, contact info
- Maintain Preferred Equine Company's professional standards
"""

reminder_manager = Agent(
    name="Reminder Coordinator",
    instructions=reminder_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini"
)

message = "Send a reminder email to John Doe about our meeting tomorrow at 2 PM"

In [25]:
tools = [tool1, tool2]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='reminder_agent1', description='Write a cold reminder email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'reminder_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000185F83A4E00>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), FunctionTool(name='reminder_agent2', description='Write a cold reminder email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'reminder_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000185F83A4540>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)]
[Agent(name='Email Mana

In [24]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

In [23]:
tools

[FunctionTool(name='reminder_subject_writer', description='Write a subject for a reminder email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'reminder_subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000185F83A6D40>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000185F83A4EA0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_outp

In [22]:
tools = [subject_tool, html_tool, send_html_email]

In [ ]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all reminder prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("monilxxxxxxx@gmail.com")  # Change to your verified sender
    to_email = To("monilxxxxx@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [20]:
subject_instructions = "This is a reminder email. \
You are given a message and you need to write a subject for a reminder email that is likely to get attention and a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Reminder Subject Writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="reminder_subject_writer", tool_description="Write a subject for a reminder email")

html_converter = Agent(name="HTML Email Converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter", tool_description="Convert a text email body to an HTML email body")

In [19]:
instructions = """
You are an employee at Preferred Equine Company. You have a meeting scheduled with a client tomorrow and need to send a reminder email.

**CLIENT INFORMATION:**
- Name: John Doe
- Email: john.doe@example.com
- Phone: 1234567890
- Address: 123 Main St, Anytown, USA
- City: Anytown
- State: CA
- Zip Code: 12345

**EMAIL REQUIREMENTS:**
1. Send a polite and professional reminder email
2. Remind the client about tomorrow's meeting
3. Include client details accurately
4. Schedule email for delivery at 10:00 AM tomorrow
5. Verify all client information is correct before sending

**EMAIL CONTENT GUIDELINES:**
- Use professional but friendly tone
- Include meeting date and time reference
- Add value by mentioning meeting agenda/preparation
- Provide contact information for any questions
- Include professional email signature

**VERIFICATION CHECKLIST:**
- Confirm client name is spelled correctly: John Doe
- Verify email address: john.doe@example.com
- Double-check all contact details
- Ensure timing: 10:00 AM delivery

"""


reminder_manager = Agent(name="Reminder Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold reminder email addressed to 'Dear Boss'"

with trace("Reminder manager"):
    result = await Runner.run(reminder_manager, message)

In [18]:
description = "Write a cold reminder email"

tool1 = reminder_agent1.as_tool(tool_name="reminder_agent1", tool_description=description)
tool2 = reminder_agent2.as_tool(tool_name="reminder_agent2", tool_description=description)

tools = [tool1, tool2, send_email]

tools

[FunctionTool(name='reminder_agent1', description='Write a cold reminder email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'reminder_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000185F83A4E00>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='reminder_agent2', description='Write a cold reminder email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'reminder_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000185F83A4540>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='se

In [17]:
tool1 = reminder_agent1.as_tool(tool_name="reminder_agent1", tool_description="Write a cold reminder email")
tool1

FunctionTool(name='reminder_agent1', description='Write a cold reminder email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'reminder_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000185F83A4220>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

In [16]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all reminder prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000185F8139E40>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

In [ ]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all reminder prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("monilxxxxxxx@gmail.com")  # Change to your verified sender
    to_email = To("monilxxxxx@gmail.com")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Reminder email", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [14]:
reminder_agent1

Agent(name='Professional Reminder Agent', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='\n                I am an employee at preferred equine company. /\n                i have a meeting with a client tomorrow. /\n                I need to send a reminder email to the client. /\n                Please write a reminder email to the client. /\n                The email should be polite and professional. /\n                The email should remind the client about the meeting. /\n                The email should be sent at 10:00 AM tomorrow. /\n                ', prompt=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, verbosity=None, metadata=None, store=None, include_usage=None, response_include=None, top_logprobs=None, extra_query=None, extra_body=None, extra_heade

In [13]:
reminder_agent1 = Agent(
        name="Professional Reminder Agent",
        instructions=instructions1,
        model="gpt-4o-mini",
)

reminder_agent2 = Agent(
        name="Verification Email Agent",
        instructions=instructions2,
        model="gpt-4o-mini",
)

In [12]:
message = "Write a cold reminder email"

with trace("Selection from remindering people"):
    results = await asyncio.gather(
        Runner.run(reminder_agent1, message),
        Runner.run(reminder_agent2, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold reminder emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    best = await Runner.run(reminder_picker, emails)

    print(f"Best remainder email:\n{best.final_output}")

Best remainder email:
Subject: Reminder: Upcoming Meeting

Dear [Client's Name],

I hope this message finds you well. I wanted to send a quick reminder about our scheduled meeting tomorrow at 10:00 AM.

Please let me know if you have any topics you would like to discuss or if there are any changes to your availability.

Looking forward to our conversation.

Best regards,

[Your Name]  
[Your Position]  
Preferred Equine Company  
[Your Contact Information]  
[Your Company Website]  


In [11]:
reminder_picker = Agent(
    name="reminder_picker",
    instructions="""You are an expert at selecting the most effective reminders. 
Your task is to choose the single best reminder from the options provided.
Consider these criteria:
1. Professionalism and tone
2. Clarity and specificity
3. Action-oriented language
4. Appropriate urgency
5. Likelihood of eliciting a response

Return only the selected reminder text, without any explanation or additional text.""",
    model="gpt-4o-mini"
)

In [10]:
message = "Write a cold remainder email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(reminder_agent1, message),
        Runner.run(reminder_agent2, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")

Subject: Reminder: Upcoming Meeting Tomorrow

Dear [Client's Name],

I hope this message finds you well. 

I wanted to send a quick reminder about our meeting scheduled for tomorrow at 10:00 AM. I look forward to discussing [briefly mention the agenda or purpose of the meeting, if applicable] and exploring how we can assist you further.

Please let me know if there are any specific topics you would like to cover or if you need to reschedule.

Thank you, and I look forward to our conversation.

Best regards,

[Your Name]  
[Your Position]  
Preferred Equine Company  
[Your Phone Number]  
[Your Email Address]  


Subject: Friendly Reminder

Hi John,

I hope this message finds you well! I wanted to follow up on our previous conversation regarding [briefly mention the topic, e.g., your project, our proposal, etc.]. 

If you have any questions or need further information, please don’t hesitate to reach out. I’m here to help!

Looking forward to your response.

Best regards,

[Your Name]  


In [9]:

from math import remainder


result = Runner.run_streamed(reminder_agent1, input="Write a cold reminder email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Friendly Reminder: Meeting Scheduled for Tomorrow

Dear [Client's Name],

I hope this message finds you well. I wanted to take a moment to remind you of our scheduled meeting tomorrow at [insert time] to discuss [briefly mention the purpose of the meeting, e.g., your needs, our services, etc.]. 

Please let me know if there are any specific topics you would like to cover or if you have any questions in advance.

I'm looking forward to our conversation!

Best regards,

[Your Name]  
[Your Position]  
Preferred Equine Company  
[Your Contact Information]  

In [7]:
reminder_agent1 = Agent(
        name="Professional Reminder Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

reminder_agent2 = Agent(
        name="Verification Email Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

In [6]:
instructions1 = """
                I am an employee at preferred equine company. /
                i have a meeting with a client tomorrow. /
                I need to send a reminder email to the client. /
                Please write a reminder email to the client. /
                The email should be polite and professional. /
                The email should remind the client about the meeting. /
                The email should be sent at 10:00 AM tomorrow. /
                """


instructions2 = """
                i need to also make sure that the email is sent to the correct client. /
                the client's name is John Doe. /
                the client's email is john.doe@example.com. /
                the client's phone number is 1234567890. /
                the client's address is 123 Main St, Anytown, USA. /
                the client's city is Anytown. /
                the client's state is CA. /
                the client's zip code is 12345. /
"""

In [ ]:
def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("monilxxxxxxx@gmail.com")  # Change to your verified sender
    to_email = To("monilxxxxx@gmail.com")  # Change to your recipient
    content = Content("text/plain", "This is an important test email")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

202


In [4]:
load_dotenv(override=True)

True

In [3]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio